In [9]:
import numpy as np
from SED_v8 import AGN
from SED_plots_v2 import Plotter
from filters import Filters
from astropy.io import fits
from astropy.io import ascii

In [10]:
ned_data = ascii.read('/Users/connor_auge/Downloads/IRAS09104_4109.csv')

In [11]:
flux_values = ned_data['Flux Density']
flux_values_array = np.array([flux_values,flux_values,flux_values,flux_values,flux_values,flux_values,flux_values,flux_values]).T

flux_values_err = ned_data['Upper limit of uncertainty']
flux_values_err_array = np.array([flux_values_err,flux_values_err,flux_values_err,flux_values_err,flux_values_err,flux_values_err,flux_values_err,flux_values_err]).T


xray_fluxes = np.array([2.4E-12,5.1E-12,1.4E-12,2.2E-12,5.0E-12,15.0E-12,6.7E-12,14.0E-12])
xray_fluxes_err = np.array([0.5E-12,1E-12,0.3E-12,0.6E-12,10E-12,3E-12,2.0E-12,6E-12])
xray_fluxes_mJy = (xray_fluxes*4.136E8/(10-2))
xray_fluxes_err_mJy = (xray_fluxes_err*4.136E8/(10-2))


In [12]:
flux_values_array[0]

array([3.84e-08, 3.84e-08, 3.84e-08, 3.84e-08, 3.84e-08, 3.84e-08,
       3.84e-08, 3.84e-08])

In [13]:
flux_array = np.array([
    xray_fluxes_mJy*1000,
    flux_values_array[1]*1E6,
    flux_values_array[2]*1E6,
    flux_values_array[3]*1E6,
    flux_values_array[4]*1E6,
    flux_values_array[5]*1E6,
    flux_values_array[6]*1E6,
    flux_values_array[7]*1E6,
    flux_values_array[8]*1E6,
    flux_values_array[9]*1E6,
    flux_values_array[10]*1E6,
    flux_values_array[11]*1E6,
    flux_values_array[12]*1E6,
    flux_values_array[13]*1E6,
    flux_values_array[14]*1E6,
    flux_values_array[16]*1E6,
    flux_values_array[18]*1E6,
])
flux_array = flux_array.T

flux_array_err = np.array([
    xray_fluxes_err_mJy*1000,
    flux_values_err_array[1]*1E6,
    flux_values_err_array[2]*1E6,
    flux_values_err_array[3]*1E6,
    flux_values_err_array[4]*1E6,
    flux_values_err_array[5]*1E6,
    flux_values_err_array[6]*1E6,
    flux_values_err_array[7]*1E6,
    flux_values_err_array[8]*1E6,
    flux_values_err_array[9]*1E6,
    flux_values_err_array[10]*1E6,
    flux_values_err_array[11]*1E6,
    flux_values_err_array[12]*1E6,
    flux_values_err_array[13]*1E6,
    flux_values_err_array[14]*1E6,
    flux_values_err_array[16]*1E6,
    flux_values_array[18]*1E6*0.45,
])
flux_array_err = flux_array_err.T

z_array = np.ones(np.shape(xray_fluxes))*0.44

model_ids = np.array(['Model1','Model2','Model3','Model4','Model5','Model6','Model7','Model8'])

filter_names = np.array(['Fx_hard','FLUX_GALEX_FUV','FLUX_GALEX_NUV','U','G','R','Z','JVHS','H_FLUX_APER2','Ks_FLUX_APER2','W1','W2','SPLASH_3_FLUX','W3','W4','IRAS3','FLUX_160',])

print(len(filter_names))
print(flux_array.shape)

17
(8, 17)


In [14]:
cigale_name = 'AHA_sample_FINAL2/Weaver_CIGALE.mag'
# cigale_name = 'test_FIR_cosmos3.mag'
inf = open(f'../xcigale/data_input/{cigale_name}', 'w')
header = np.asarray(['# id', 'redshift'])
cigale_filters = Filters('filter_list.dat').pull_filter(filter_names, 'xcigale name')
for i in range(len(cigale_filters)):
    header = np.append(header, cigale_filters[i])
    header = np.append(header, cigale_filters[i]+'_err')
np.savetxt(inf, header, fmt='%s', delimiter='    ', newline=' ')
inf.close()

In [15]:

for i in range(len(model_ids)):

    source = AGN(model_ids[i],z_array[i],filter_names,flux_array[i],flux_array_err[i])
    source.MakeSED()

    make_dict = source.MakeDict(filter_names,flux_array[i])
    make_dict_err = source.MakeDict(filter_names,flux_array_err[i])

    f1 = source.Find_value(1.0)

    # ix, iy = source.Int_SED(xmin=1E-1, xmax=1E10)
    # wfir, ffir, f100 = source.Int_SED_FIR(Find_value=100.0,discreet=True)
    # Id, redshift, w, f, frac_err = source.pull_plot_info(norm_w=1)
    # plot = Plotter(Id, redshift, w, f, 44.5,f1,np.nan)

    # plot.PlotSED()

    source.write_cigale_file2(cigale_name, filter_names, make_dict, make_dict_err, int_fx=xray_fluxes_mJy[i], int_fx_err=xray_fluxes_err_mJy[i])

